In [1]:
import mysql.connector
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random
from datetime import datetime

In [2]:
passcode = input("What is your password?: ")

In [4]:
def fetch_patient_info(passcode):
    # Establish connection to MySQL Database
    connection = mysql.connector.connect(
        host="localhost",
        user="root",
        password=passcode,
        database="mock_patient_project"
    )
    # print("Connected to MySQL Database")

    # Create cursor
    cursor = connection.cursor()

    try:
        # Query random medical record number from patient_data table
        query_medical_record_data = "SELECT * FROM patient_data ORDER BY RAND() LIMIT 1"
        cursor.execute(query_medical_record_data)
        medical_record_data = cursor.fetchone()

        # Get column names from cursor description
        column_names = [description[0] for description in cursor.description]

        # Construct DataFrame with appropriate column labels
        patient_info_df = pd.DataFrame([medical_record_data], columns=column_names)

        # Print the DataFrame
        # print("Patient Info DataFrame:")
        # print(patient_info_df)

    finally:
        # Close cursor and connection
        cursor.close()
        connection.close()
        # print("MySQL connection is closed")

    return patient_info_df

In [5]:
def generate_percentage():
    # Generate a random number from a normal distribution centered around 0.15
    # with a standard deviation that controls the spread of the bell curve
    mean = 0.015
    std_dev = 0.005  # Adjust this value to control the spread of the bell curve
    percentage = np.random.normal(mean, std_dev)
    
    # Ensure the generated percentage falls within the desired range [0.1, 0.2]
    percentage = max(0.01, min(0.02, percentage))
    
    return percentage

# Call the function to generate the percentage
# percentage_to_fetch = generate_percentage()

# print("Generated Percentage:", percentage_to_fetch)


In [6]:
def determine_reason_for_encounter(age, gender):
    if age <= 18:
        reasons = [
            "Infectious diseases",
            "Accidents and injuries",
            "Childhood illnesses",
            "Appendicitis"
        ]
    elif age <= 64:
        reasons = [
            "Pregnancy and childbirth" if age < 35 and gender == "Female" and random.random() < (35 - age) / 35 else None,
            "Accidents and injuries",
            "Chronic illnesses",
            "Mental health concerns",
            "Surgery"
        ]
    else:
        reasons = [
            "Chronic illnesses",
            "Falls and fractures",
            "Pneumonia",
            "Complications from surgery"
        ]

    # Filter out None values and select a random reason
    reasons = [reason for reason in reasons if reason is not None]
    selected_reason = random.choice(reasons) if reasons else "Reason not specified"

    return selected_reason


In [7]:
# Set the percentage of records to fetch
percentage_to_fetch = generate_percentage()  # Change this percentage as needed
num_iterations = 10  # Number of iterations
# print("Generated Percentage:", percentage_to_fetch)

# Establish connection to MySQL Database
connection = mysql.connector.connect(
    host="localhost",
    user="root",
    password=passcode,
    database="mock_patient_project"
)
cursor = connection.cursor()

# Get total number of records in patient_data table
cursor.execute("SELECT COUNT(*) FROM patient_data")
total_records = cursor.fetchone()[0]
# print("Total Records in Patient Data Table:", total_records)

# Close cursor and connection
cursor.close()
connection.close()

# Calculate the number of records to fetch based on the percentage
num_records_to_fetch = int(total_records * (percentage_to_fetch / 100))

# Create empty list to store patient info dataframes
patient_info_dfs = pd.DataFrame()

# Iterate over the number of iterations
for i in range(num_records_to_fetch):
    # Call the function to fetch patient info
    patient_info_df = fetch_patient_info(passcode)

    # Determine reason for encounter based on age and gender
    age = patient_info_df['Age'].iloc[0]
    gender = patient_info_df['Gender'].iloc[0]
    reason_for_encounter = determine_reason_for_encounter(age, gender)

    # Add reason for encounter to DataFrame
    patient_info_df['ReasonForEncounter'] = reason_for_encounter

    # Add today's date to the DataFrame
    today_date = datetime.now().strftime("%Y-%m-%d")
    patient_info_df['EncounterDate'] = today_date

    # Append the returned dataframe to the list
    patient_info_dfs = pd.concat([patient_info_dfs, patient_info_df], ignore_index=True)

# Print the combined DataFrame
print(patient_info_dfs)

   FirstName  LastName              Name DateOfBirth  Age  Gender  \
0       Mary    Wagner       Mary Wagner  1968-09-01   24    Male   
1       Lisa    Bolton       Lisa Bolton  1993-08-11   18    Male   
2       Evan     Cantu        Evan Cantu  1940-04-24   40  Female   
3   Kimberly   Carroll  Kimberly Carroll  1942-01-13   45    Male   
4      David    Garcia      David Garcia  1963-01-04    9  Female   
5    Jasmine      Boyd      Jasmine Boyd  1998-05-13   37  Female   
6   Margaret      Ross     Margaret Ross  1938-07-29   54    Male   
7       Carl   Chapman      Carl Chapman  1997-04-12   11    Male   
8     Willie  Mitchell   Willie Mitchell  1973-05-19    3  Female   
9     Andrea    Taylor     Andrea Taylor  1982-04-08   56    Male   
10    Laurie   Johnson    Laurie Johnson  1957-04-19   61    Male   

   MedicalRecordNumber      ReasonForEncounter EncounterDate  
0      E143255861193\r                 Surgery    2024-04-02  
1      E145634474163\r  Accidents and injurie

In [ ]:
# # Function to generate encounters from past dates
# def generate_past_encounters(num_encounters):
#     encounters = []
#     today = datetime.now().date()
#     for _ in range(num_encounters):
#         # Generate a random date within the past 5 years
#         encounter_date = today - timedelta(days=random.randint(1, 1825))  # 5 years * 365 days = 1825 days
#         encounters.append(encounter_date)
#     return encounters

# # Set the percentage of records to fetch
# percentage_to_fetch = generate_percentage()  # Change this percentage as needed
# num_iterations = 10  # Number of iterations
# num_records_to_fetch = 3500  # Number of encounters from past dates
# print("Generated Percentage:", percentage_to_fetch)

# # Generate past encounter dates
# past_encounters = generate_past_encounters(num_records_to_fetch)

# # Create empty list to store patient info dataframes
# patient_info_dfs = pd.DataFrame()

# # Iterate over the number of iterations
# for i in range(num_records_to_fetch):
#     # Print the current iteration number
#     print("Iteration:", i+1)  
    
#     # Randomly select a past encounter date
#     encounter_date = random.choice(past_encounters)

#     # Call the function to fetch patient info
#     patient_info_df = fetch_patient_info()

#     # Determine reason for encounter based on age and gender
#     age = patient_info_df['Age'].iloc[0]
#     gender = patient_info_df['Gender'].iloc[0]
#     reason_for_encounter = determine_reason_for_encounter(age, gender)

#     # Add reason for encounter and encounter date to DataFrame
#     patient_info_df['ReasonForEncounter'] = reason_for_encounter
#     patient_info_df['EncounterDate'] = encounter_date

#     # Append the returned dataframe to the list
#     patient_info_dfs = pd.concat([patient_info_dfs, patient_info_df], ignore_index=True)

# # Print the combined DataFrame
# print(patient_info_dfs)


In [ ]:
def table_exists(cursor, table_name):
    cursor.execute("SHOW TABLES LIKE %s", (table_name,))
    return cursor.fetchone() is not None

In [ ]:
def upload_to_mysql(dataframe, table_name, passcode):
    # Establish connection to MySQL Database
    connection = mysql.connector.connect(
        host="localhost",
        user="root",
        password=passcode,
        database="mock_patient_project"
    )
    cursor = connection.cursor()

    try:
        # Check if table exists
        # if not table_exists(cursor, table_name):
        #     print(f"Table '{table_name}' does not exist.")
        #     return
        # print(f"Table '{table_name}' exist.")
        
        # Iterate over each row in the DataFrame and insert it into the MySQL table
        for index, row in dataframe.iterrows():
            # Prepare column names string
            columns_str = ', '.join(dataframe.columns)

            # Prepare placeholders for data values
            placeholders = ', '.join(['%s'] * len(row))

            # Construct SQL query for row insertion
            sql = f"INSERT INTO {table_name} ({columns_str}) VALUES ({placeholders})"

            # print("Executing SQL:", sql)  # Print SQL query being executed

            # Execute SQL query with row data
            cursor.execute(sql, tuple(row))

            # print("Row inserted successfully.")  # Print success message

        # Commit changes
        connection.commit()
        # print("Changes committed successfully.")  # Print success message

    except Exception as e:
        print("Error during insertion:", e)
        connection.rollback()
        print("Changes rolled back.")  # Print rollback message

    finally:
        # Close cursor and connection
        cursor.close()
        connection.close()

In [ ]:
upload_to_mysql(patient_info_dfs, 'encounters', passcode)